In [1]:
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import pandas as pd
import os
import csv
from scipy.stats import pearsonr
import numpy as np
import re
import pytrends
import bash
import random
import pdb
import calendar
import requests
from requests.exceptions import Timeout
from pytrends.exceptions import ResponseError
from torpy.circuit import  CellTimeoutError

In [2]:
from pytrends.request import TrendReq

requests_args = {
    'headers' : {
        'authority': 'trends.google.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en;q=0.9,zh-TW;q=0.8,zh;q=0.7,en-US;q=0.6,zh-CN;q=0.5',
        'cookie': '__utmc=10102256; __utmz=10102256.1678192450.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utma=10102256.2025600903.1678192450.1678195972.1678286166.3; SEARCH_SAMESITE=CgQI25YB; HSID=AUQLq4lSl8JHUmfuc; SSID=AGePUZzi4zshU-OBF; APISID=bd0oYBEnVeUMWngv/AwhBQIWpjgLUcn3XC; SAPISID=z23guNaQsrhC27n1/AdWg_wlI7HNDhnlCZ; __Secure-1PAPISID=z23guNaQsrhC27n1/AdWg_wlI7HNDhnlCZ; __Secure-3PAPISID=z23guNaQsrhC27n1/AdWg_wlI7HNDhnlCZ; __Secure-3PSID=TwiACySIKo2Fm3zRTTS2OuPqFN_aCVnyuecv3eLaPV_TBw6Oz_6VanMNXNoSGhV7Xgr0Qw.; __Secure-1PSID=TwiACySIKo2Fm3zRTTS2OuPqFN_aCVnyuecv3eLaPV_TBw6OS4BhSqBIMdv6ekc8PtvkJw.; SID=TwiACySIKo2Fm3zRTTS2OuPqFN_aCVnyuecv3eLaPV_TBw6OOS7vAi4RnJN0z452tM-dTA.; AEC=ARSKqsLfhZiy7Ti_OicTHlHiiiamiAh-7u0hHtqGAeknIzYKp8WFLuKTFg; 1P_JAR=2023-03-12-11; NID=511=tjpTaOk3jvuzWaDZa4b3P83oSd-Lo0te7UENpaGpDpGzB1tNDl89ts4ej4b1jOvGh9PTG5j-31sv4eMbzGpZ_fh5pnggrmWIl0tbZMF1akwUNWFHNVGy2N1C4Df-tC1EW2_mwhcsXcBv9-ZXlImvK7Lg42tRKWmi5rVYFevF4kFXA2kxtt4g4Ara0gpUmJAAK4NHsjIWRaAKkLoXZdTBGlGDqB5SvDwKJjHPWu37REXSGI4Ov4ixist7SeX_UkrxlxiZwfEMZjdMHq9lZzbKcAVRTL8UUdRT; _ga=GA1.3.2144743692.1678192451; _gid=GA1.3.1517861668.1678621585; _gat_gtag_UA_4401283=1; OTZ=6938626_72_76_104100_72_446760; SIDCC=AFvIBn_Cvl17eSDEOyESFOBvsZ_uoLC9j1TSUvv0J3bNZRTtoRw95k747ORaLwquXQXAJr8qeQ; __Secure-1PSIDCC=AFvIBn92qPNq5AqpBVV_QnlYZIswhlzM1qGP-sfnSsKm6-thCEaCisScvnbrJr1ex3zy0lHgnls; __Secure-3PSIDCC=AFvIBn8P6d-liUVt73mEM1idcNadmpzHswFTRE5ci8LZr7r7yV6aehlPKBjHPbmRguP5Z3rV4Q; _ga_VWZPXDNJJB=GS1.1.1678621584.6.1.1678621638.0.0.0',
        'referer': 'https://trends.google.com/',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-arch': '"x86"',
        'sec-ch-ua-bitness': '"64"',
        'sec-ch-ua-full-version': '"111.0.5563.64"',
        'sec-ch-ua-full-version-list': '"Google Chrome";v="111.0.5563.64", "Not(A:Brand";v="8.0.0.0", "Chromium";v="111.0.5563.64"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-model': '""',
        'sec-ch-ua-platform': '"Windows"',
        'sec-ch-ua-platform-version': '"10.0.0"',
        'sec-ch-ua-wow64': '?0',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'x-client-data': 'CIa2yQEIpbbJAQjEtskBCKmdygEIqd3KAQiUocsBCOeyzAEI6fXMAQjJ/8wBCI+MzQEI9pPNAQjwlc0BCKeWzQEI4pfNAQjjl80BCM2YzQEIuZnNAQj0mc0BCLSazQEYwcvMAQ==',
    }
}
 
    

In [3]:
def init():
    #https://pypi.org/project/torpy/
    #pip install torpy
    from torpy import TorClient
    hostname = 'ifconfig.me'  # It's possible use onion hostname here as well
    with TorClient() as tor:
        # Choose random guard node and create 3-hops circuit
        with tor.create_circuit(3) as circuit:
            # Create tor stream to host
            with circuit.create_stream((hostname, 80)) as stream:
                # Now we can communicate with host
                stream.send(b'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % hostname.encode())
                recv = stream.recv(1024)
    return 0

In [4]:
def column_name_to_mvt_and_remove_zero_values(df, geo_name):
    df.rename(columns={df.columns[0]: f"MVT_GT_{geo_name}" }, inplace = True)
    df = df[df[f"MVT_GT_{geo_name}"] != 0]
    return df

In [5]:
def execute_init():
    while True:
        try:
            init()
            print("init status completed")
            break
        except (TimeoutError, CellTimeoutError, ConnectionResetError) as error:
            print(f"Init Error Occured: {error}")
            print("Let me sleep for 10 seconds")
            print("ZZzzzz...")
            time.sleep(10)
            print("Was a nice sleep, now let me continue...")
            pass
    return 0

In [6]:
def GT_temp_df(start_date, end_date, geo_code, geo_name):
    
    temp_df = pd.DataFrame()
    while len(temp_df) == 0:
        try:
            pytrend.build_payload(mvt_keyword, cat=0, timeframe=f"{start_date} {end_date}", geo = geo_code)
            temp_df = pytrend.interest_over_time()

        except (ResponseError, Timeout, ConnectionAbortedError, ConnectionError) as error:
            try:
                print(f"GT Error Occured: {error}")
                !pip install pytrends --upgrade
                #init()
                #rint(f"Let's Go, Onion!")
                pytrend.build_payload(mvt_keyword, cat=0, timeframe=f"{start_date} {end_date}", geo = geo_code)
                temp_df = pytrend.interest_over_time()

            except (ResponseError, Timeout, ConnectionAbortedError, ConnectionError) as error:
                print(f"GT Error Occured: {error}")
                print(f"Let me sleep for 1 day, till {(datetime.now() + timedelta(days=1)).strftime('%m/%d/%Y, %H:%M:%S')}")
                print("ZZzzzz...")
                time.sleep(random.randint(60*60*24, 60*60*24+30,))
                print("Was a nice sleep, now let me continue...")


    print(f"Obtaining GT MVT from {start_date} to {end_date} in {geo_name} metro area")
    time.sleep(random.randint(120, 130))
    temp_df = column_name_to_mvt_and_remove_zero_values(temp_df, geo_name)
    return temp_df

In [7]:
def combine_df_and_temp_df_and_rescale_by_overlap_factors(df, temp_df):
    df = pd.concat([df, temp_df.iloc[:, 0]], axis = 1)
    if len(df.columns) > 1:
        factor_data = df.dropna(how = "any")
        factor_divided = factor_data.iloc[:, 0]/factor_data.iloc[:, 1]
        adjusted_factor = factor_divided.mean()
        df.iloc[:,1] = df.iloc[:,1]*adjusted_factor
        df = df.mean(axis = 1)
    return df

In [8]:
#init()

In [9]:
#basic pytrend settings
pytrend = TrendReq(requests_args=requests_args,  retries=3)

In [10]:
'''
kw_list = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check",
    "I raped+raped me+raped report police+being raped+been raped - dream - kobe - trump - porn",
    "my stolen+find my stolen+theft report police+someone stolen my - car - heart - dream - number",
    "burglary+home burglarized+my house broken into+my home broken into+burglary report police - dream",
    "my committed a crime+police arrested my+my arrested+my in jail+my in prison - arrested development",
    "I hate family+parents hate me+I hate my parents+I hate school+I hate teacher+I hate dad+I hate mom"]
'''
#years = [i for i in range(2010, 2016)]
#print(years)
#months = [i for i in range(1, 13, 3)] 
#print(months)

'\nkw_list = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check",\n    "I raped+raped me+raped report police+being raped+been raped - dream - kobe - trump - porn",\n    "my stolen+find my stolen+theft report police+someone stolen my - car - heart - dream - number",\n    "burglary+home burglarized+my house broken into+my home broken into+burglary report police - dream",\n    "my committed a crime+police arrested my+my arrested+my in jail+my in prison - arrested development",\n    "I hate family+parents hate me+I hate my parents+I hate school+I hate teacher+I hate dad+I hate mom"]\n'

### GT Keywords

In [11]:
mvt_keyword = ["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check"] 

### Geo Location and Geo Codes

In [12]:
# Top 50 the most populous cities in the U.S.
region_code_dict = {
    "US-NY-501": "New York NY",
    "US-CA-807": "San Francisco-Oakland-San Jose CA", #including Oakland, CA and San Jose CA
    "US-CA-803": "Los Angeles CA", #including Long Beach, CA
    "US-CA-825": "San Diego CA", 
    "US-CA-862": "Sacramento-Stockton-Modesto CA",
    "US-AZ-753": "Phoenix AZ", #including Mesa, AZ
    "US-MI-505": "Detroit MI",
    "US-NC-517" : "Charlotte NC",
    "US-OH-535" : "Columbus OH",
    "US-FL-561" : "Jacksonville FL",
    "US-MD-511" : "Washington DC (Hagerstown MD)", # including Arlington VA
    "US-OK-650" : "Oklahoma City OK",
    "US-TX-765" : "El Paso TX",
    "US-OR-820" : "Portland OR",
    "US-MD-512" : "Baltimore MD",
    "US-WI-617" : "Milwaukee WI",
    "US-NM-790" : "Albuquerque-Santa Fe NM",
    "US-CA-866" : "Fresno-Visalia CA",
    "US-MO-616" : "Kansas City MO",
    "US-NE-652" : "Omaha NE",
    "US-CO-752" : "Colorado Springs-Pueblo CO",
    "US-NC-560" : "Raleigh-Durham (Fayetteville) NC",
    "US-VA-544" : "Norfolk-Portsmouth-Newport News VA", #including Virginia Beach, VA
    "US-FL-528" : "Miami-Ft. Lauderdale FL",
    "US-CA-800" : "Bakersfield CA",
    "US-OK-671" : "Tulsa OK",
    "US-KS-678" : "Wichita-Hutchinson KS"  ,
    "US-NV-839":"Las Vegas NV",
    "US-WA-819":"Seattle-Tacoma WA",
    "US-CO-751":"Denver CO", #including Aurora, CO
    "US-PA-504":"Philadelphia PA",
    "US-GA-524":"Atlanta GA",
    "US-IN-602":"Chicago IL",
    "US-NH-506":"Boston MA-Manchester NH",
    "CA-ON":"Ontario",
    "US-KY-529" : "Louisville KY",
    "US-TN-640" : "Memphis TN",
    "US-WI-613" : "Minneapolis-St. Paul MN",
    "US-TN-659" : "Nashville TN",
    "US-AZ-789" : "Tucson AZ",
    "US-TX-623" : "Dallas-Ft. Worth TX", # including Fort Worth, TX
    "US-TX-641" : "San Antonio TX",
    "US-TX-618" : "Houston TX",
    "US-TX-635" : "Austin TX",
}






### Dates (From, To)

In [13]:
From  = ["2017-01-01",
         "2018-01-01"]

To = ["2021-12-31",
      "2022-12-31"]

In [14]:
To[-1]

'2022-12-31'

In [15]:
From[0]

'2017-01-01'

## Execute Pytrends to Pull Daily Data from Google Trends

In [ ]:

#Get 100 times weekly GT MVT data per city
for times in range(100):
    for geo_code, geo_name in zip(region_code_dict.keys(), region_code_dict.values()):
        df = pd.DataFrame()

        for start_date, end_date in zip(From, To):
            #Get daily estimates with adjusted factors [mean(old/new) * new]
            temp_df = GT_temp_df(start_date, end_date, geo_code, geo_name)
            df = combine_df_and_temp_df_and_rescale_by_overlap_factors(df, temp_df)

        df.name = f"MVT_GT_{geo_name}"
        df.to_csv(f"{geo_name}_weekly_{From[0]}_{To[-1]}_{pd.Timestamp.now().strftime('%Y%m%d_%H_%M')}.csv")
        print(f"{geo_name}_weekly_{From[0]}_{To[-1]}_{pd.Timestamp.now().strftime('%Y%m%d_%H_%M')} file saved")

Obtaining GT MVT from 2017-01-01 to 2021-12-31 in New York NY metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in New York NY metro area
New York NY_weekly_2017-01-01_2022-12-31_20230314_19_48 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in San Francisco-Oakland-San Jose CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in San Francisco-Oakland-San Jose CA metro area
San Francisco-Oakland-San Jose CA_weekly_2017-01-01_2022-12-31_20230314_19_52 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Los Angeles CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Los Angeles CA metro area
Los Angeles CA_weekly_2017-01-01_2022-12-31_20230314_19_56 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in San Diego CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in San Diego CA metro area
San Diego CA_weekly_2017-01-01_2022-12-31_20230314_20_00 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Sacramento-Stockton-

Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Memphis TN metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Memphis TN metro area
Memphis TN_weekly_2017-01-01_2022-12-31_20230314_22_19 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Minneapolis-St. Paul MN metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Minneapolis-St. Paul MN metro area
Minneapolis-St. Paul MN_weekly_2017-01-01_2022-12-31_20230314_22_23 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Nashville TN metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Nashville TN metro area
Nashville TN_weekly_2017-01-01_2022-12-31_20230314_22_27 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Tucson AZ metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Tucson AZ metro area
Tucson AZ_weekly_2017-01-01_2022-12-31_20230314_22_31 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Dallas-Ft. Worth TX metro area
Obtaining GT MVT from 2018-01-01 to 2

Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Seattle-Tacoma WA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Seattle-Tacoma WA metro area
Seattle-Tacoma WA_weekly_2017-01-01_2022-12-31_20230315_00_49 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Denver CO metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Denver CO metro area
Denver CO_weekly_2017-01-01_2022-12-31_20230315_00_53 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Philadelphia PA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Philadelphia PA metro area
Philadelphia PA_weekly_2017-01-01_2022-12-31_20230315_00_57 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Atlanta GA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Atlanta GA metro area
Atlanta GA_weekly_2017-01-01_2022-12-31_20230315_01_01 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Chicago IL metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Chica

Colorado Springs-Pueblo CO_weekly_2017-01-01_2022-12-31_20230315_03_19 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Raleigh-Durham (Fayetteville) NC metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Raleigh-Durham (Fayetteville) NC metro area
Raleigh-Durham (Fayetteville) NC_weekly_2017-01-01_2022-12-31_20230315_03_23 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Norfolk-Portsmouth-Newport News VA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Norfolk-Portsmouth-Newport News VA metro area
Norfolk-Portsmouth-Newport News VA_weekly_2017-01-01_2022-12-31_20230315_03_28 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Miami-Ft. Lauderdale FL metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Miami-Ft. Lauderdale FL metro area
Miami-Ft. Lauderdale FL_weekly_2017-01-01_2022-12-31_20230315_03_32 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Bakersfield CA metro area
Obtaining GT MVT from 2018-01-01 to

El Paso TX_weekly_2017-01-01_2022-12-31_20230315_05_50 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Portland OR metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Portland OR metro area
Portland OR_weekly_2017-01-01_2022-12-31_20230315_05_54 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Baltimore MD metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Baltimore MD metro area
Baltimore MD_weekly_2017-01-01_2022-12-31_20230315_05_59 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Milwaukee WI metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Milwaukee WI metro area
Milwaukee WI_weekly_2017-01-01_2022-12-31_20230315_06_03 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Albuquerque-Santa Fe NM metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Albuquerque-Santa Fe NM metro area
Albuquerque-Santa Fe NM_weekly_2017-01-01_2022-12-31_20230315_06_07 file saved
Obtaining GT MVT from 2017-01-01 to 2021

...

Norfolk-Portsmouth-Newport News VA_weekly_2017-01-01_2022-12-31_20230316_07_06 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Miami-Ft. Lauderdale FL metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Miami-Ft. Lauderdale FL metro area
Miami-Ft. Lauderdale FL_weekly_2017-01-01_2022-12-31_20230316_07_10 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Bakersfield CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Bakersfield CA metro area
Bakersfield CA_weekly_2017-01-01_2022-12-31_20230316_07_14 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Tulsa OK metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Tulsa OK metro area
Tulsa OK_weekly_2017-01-01_2022-12-31_20230316_07_18 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Wichita-Hutchinson KS metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Wichita-Hutchinson KS metro area
Wichita-Hutchinson KS_weekly_2017-01-01_2022-12-31_20230316_07_22 fil

Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Milwaukee WI metro area
Milwaukee WI_weekly_2017-01-01_2022-12-31_20230316_09_41 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Albuquerque-Santa Fe NM metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Albuquerque-Santa Fe NM metro area
Albuquerque-Santa Fe NM_weekly_2017-01-01_2022-12-31_20230316_09_45 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Fresno-Visalia CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Fresno-Visalia CA metro area
Fresno-Visalia CA_weekly_2017-01-01_2022-12-31_20230316_09_49 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Kansas City MO metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Kansas City MO metro area
Kansas City MO_weekly_2017-01-01_2022-12-31_20230316_09_53 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Omaha NE metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Omaha NE metro area
Omaha NE_weekly_

Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Charlotte NC metro area
Charlotte NC_weekly_2017-01-01_2022-12-31_20230316_12_11 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Columbus OH metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Columbus OH metro area
Columbus OH_weekly_2017-01-01_2022-12-31_20230316_12_16 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Jacksonville FL metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Jacksonville FL metro area
Jacksonville FL_weekly_2017-01-01_2022-12-31_20230316_12_20 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Washington DC (Hagerstown MD) metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Washington DC (Hagerstown MD) metro area
Washington DC (Hagerstown MD)_weekly_2017-01-01_2022-12-31_20230316_12_24 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Oklahoma City OK metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Oklahoma City OK metro ar

Austin TX_weekly_2017-01-01_2022-12-31_20230316_14_42 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in New York NY metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in New York NY metro area
New York NY_weekly_2017-01-01_2022-12-31_20230316_14_46 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in San Francisco-Oakland-San Jose CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in San Francisco-Oakland-San Jose CA metro area
San Francisco-Oakland-San Jose CA_weekly_2017-01-01_2022-12-31_20230316_14_50 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Los Angeles CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Los Angeles CA metro area
Los Angeles CA_weekly_2017-01-01_2022-12-31_20230316_14_54 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in San Diego CA metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in San Diego CA metro area
San Diego CA_weekly_2017-01-01_2022-12-31_20230316_14_58 file saved
Obtai

Louisville KY_weekly_2017-01-01_2022-12-31_20230316_17_12 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Memphis TN metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Memphis TN metro area
Memphis TN_weekly_2017-01-01_2022-12-31_20230316_17_16 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Minneapolis-St. Paul MN metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Minneapolis-St. Paul MN metro area
Minneapolis-St. Paul MN_weekly_2017-01-01_2022-12-31_20230316_17_20 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Nashville TN metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Nashville TN metro area
Nashville TN_weekly_2017-01-01_2022-12-31_20230316_17_25 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in Tucson AZ metro area
Obtaining GT MVT from 2018-01-01 to 2022-12-31 in Tucson AZ metro area
Tucson AZ_weekly_2017-01-01_2022-12-31_20230316_17_29 file saved
Obtaining GT MVT from 2017-01-01 to 2021-12-31 in

In [ ]:
test.plot(figsize=(20, 12),  kind ='line')

In [ ]:
[print(i/25*23) for i in test2["car stolen+find stolen car+report police stolen car+insurance car stolen-dream-check"]]

In [ ]:
kw_list=['Facebook', 'Apple', 'Amazon', 'Netflix', 'Google']
pytrend = TrendReq()
#search interest per region
#run model for keywords (can also be competitors)
pytrend.build_payload(kw_list, timeframe='today 1-m')

# Interest by Region
regiondf = pytrend.interest_by_region()
#looking at rows where all values are not equal to 0
regiondf = regiondf[(regiondf != 0).all(1)]

#drop all rows that have null values in all columns
regiondf.dropna(how='all',axis=0, inplace=True)

#visualise
regiondf.plot(figsize=(20, 12), y=kw_list, kind ='bar')

In [ ]:
historicaldf = pytrend.get_historical_interest(['Facebook', 'Apple', 'Amazon', 'Netflix', 'Google'], 
                                               year_start=2020, month_start=1, day_start=1, hour_start=0, 
                                               year_end=2020, month_end=12, day_end=31, hour_end=0, 
                                               cat=0, geo='en-US', gprop='', sleep=0)

#visualise
#plot a timeseries chart
historicaldf.plot(figsize=(20, 12))

#plot seperate graphs, using theprovided keywords
historicaldf.plot(subplots=True, figsize=(20, 12))

In [ ]:
def gt_func(start_date_list, end_date_list, resolution, sample_size, path):
#get monthly GT data
    import inflect
    #give ordinal names to numbers
    p = inflect.engine()

    for t in range(sample_size): #sample from GT 100 times
        #time.sleep(random.randint(21600,21668)) # 6 hrs
        print(f"The {p.ordinal(t+1)} pull from GT")
        qt_list = ["Q1","Q2","Q3","Q4"]
        count = 0
        for i, j in zip(start_date_list, end_date_list):
            df = pd.DataFrame()
            #Q1 - Q4
            if count == 4:
                count = 0
            for k in kw_list:
                #q = the name of the quarter
                q = qt_list[count]
                try:
                    pytrend.build_payload(k, cat=0, timeframe=f'{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}',  geo='US', gprop='')
                    df = pd.concat([df, pytrend.interest_by_region(resolution="DMA", inc_low_vol = False, inc_geo_code = False)], axis = 1)
                    time.sleep(random.randint(120, 130))

                except (Timeout, ResponseError) as e:
                        print(f'!!{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}!! ReadTimeoutError')
                        print(f"Whoops! ReadTimeoutError, the {p.ordinal(t+1)} pull aborted, now we take a rest")
                        print(dt.datetime.now().strftime("%Y/%m/%d_%H:%M:%S"))
                        time.sleep(random.randint(86668, 86888)) # 24 hrs
                        print(f"Rest enough, now we continue!")

                        #continue working
                        pytrend.build_payload(k, cat=0, timeframe=f'{i.strftime("%Y-%m-%d")} {j.strftime("%Y-%m-%d")}',  geo='US', gprop='')
                        df = pd.concat([df, pytrend.interest_by_region(resolution="DMA", inc_low_vol=False, inc_geo_code=False)], axis = 1)
                        time.sleep(random.randint(120, 130))

            df.columns = [f"MVT_{i.strftime('%Y')}_{q}", 
                          f"rape_{i.strftime('%Y')}_{q}", 
                          f"larceny_{i.strftime('%Y')}_{q}", 
                          f"burglary_{i.strftime('%Y')}_{q}", 
                          f"known_deliq_{i.strftime('%Y')}_{q}", 
                          f"low_social_cont_{i.strftime('%Y')}_{q}"]
            df.to_csv(f'{path}gt_crime_{i.strftime("%Y")}_{q}_{dt.datetime.now().strftime("%Y%m%d_%H-%M-%S")}.csv')
            count += 1
        print(f"The {p.ordinal(t+1)} pull from GT Done!!")
        print(f'Time finished: {dt.datetime.now().strftime("%Y/%m/%d, %H:%M")}')
        time.sleep(random.randint(10800,10868)) # 3 hrs


    print(f"Congratulations, Well Done!")